In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# hyperparameters
block_size=8
batch_size=4
max_iters = 1000
# eval_interval
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cpu


In [44]:
# open txt file and read the content into text var
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [45]:
# character level tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [46]:
# train and validation splits
n =int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
# print(x.shape)
print('targets:')
print(y)

inputs:
tensor([[67, 57,  1, 54, 72,  1, 60, 71],
        [58, 10, 56, 71, 78,  1, 73, 61],
        [67, 73, 72,  1, 76, 68, 67,  5],
        [67, 60,  1, 54, 73,  1, 61, 58]])
targets:
tensor([[57,  1, 54, 72,  1, 60, 71, 58],
        [10, 56, 71, 78,  1, 73, 61, 58],
        [73, 72,  1, 76, 68, 67,  5, 73],
        [60,  1, 54, 73,  1, 61, 58, 71]])


In [47]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [48]:
# inputs and target implementation
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f'when input is {context} target is {target}')

In [49]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:]  # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample form the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled inex to the running sequence 
            index = torch.cat((index, index_next), dim=1)  # (B,T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


1Jqb_v628)ccC:XVZZGdbhUIwdQz?4IwINHgK[,3IQRzpel]R*)Hi7_0PWwF."xM4oB?elMojHXNy4cIwoc"IYdtv'_ZZr_OlDClR,6relops!lrjEgJp ke5(F.G
Fqh5QgNq
JhSAg(O6]NIS;2)rNnjP"3hf'kFg2z"ZlIpD,)mQrNEZKkwOj(oz&"lhY9[GFL,6
1hHme3s'ClIPWJ:Qk',elXBlX6mQF;,qLTq]t"*;OA!le)&b7wKSdWQZi*WQRn'Vw1MwGqTU
1zwt;5(iMIq 4tjkI5o8skyY.R]sKk(I-1ymGMB?[nOUCi,nts9&Nmb*4e9J'yuqZsD[S]tw,lx0T9cVy'xNmOl[S!aHTz&m[?a
yF1pj15,(OZ:L*1yz"tLpy M(2lNiiUH*2w&m-::l0Jnj,E:,yX3b.yE,rN)rhLEs??B5NeYRWJi.,[K9Eg;w]*4Mt'-kxzpyb7.Ih!CNQ23p3R5iA*F!qVO._IUQVZ


In [51]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.844, val loss: 4.854
step: 250, train loss: 4.785, val loss: 4.807
step: 500, train loss: 4.717, val loss: 4.740
step: 750, train loss: 4.662, val loss: 4.678
4.668766498565674


In [33]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


f(nnimCd tindxWst3)Nw0YfCbLnos;GQ35AKFogedLnd 7[AOh(CDoroelQ;Wim.*
yhd 6uoC]5uL5.R:( cPtolk?3PLkgVYhah w,  fhOFUain "Waan]40d d [:R6, 4xGut
**e[)J9otgugD_ou,exWJ*iis'7)oid bgZFHW_QJBWp Wisc)Blug
70h0nOFm "A-Vclffoio-"ZofIMv0qW7iW_VS]Ca; t1njPl
yU5E
nt;:Y80I1iXXSTulyjC'c9pLCIDL_Kgy:7Y7_r
y_Dgheen5XXXGud.sth(mNAB"anugR*a tld, thi
c9"bLncFZ(4Rn?;Hd3O4Gp stKkaze_bLhanoIlN diis﻿Raz
WDJ&5XXCXQ*v9AP1me-
6st w5EB[Hd!Curof
L﻿z
S)orv(timua s
kX9N5dovyjpahr:esn]]b-xIP5)UGttXQ"0NvvjrrB﻿iWAranto Be pzce
KL7W
